In [1]:
from polarform import extractAmplitudeAndPhase
from dft import fft, phaseIFFT, ampIFFT, oneDomainIFFT
import imgmgmt as img
from pointop import contrastStretching, normalizeLog
from pgmf import readPGM, writePGM

In [2]:
file = open('./1/Cross.pgm', 'rb')
depth, cross = readPGM(file)

In [3]:
cross = img.inverse(cross)
img.padding(cross, 56)
cross = img.inverse(cross)
w = len(cross[0])
h = len(cross)

In [4]:
# 1.1
crossFT = fft(cross)
cross_amplitude, cross_phase = extractAmplitudeAndPhase(crossFT)

w, h, contrast = contrastStretching(cross_phase)
file = open('./1/CrossPhase.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(contrast))\

w, h, norm = normalizeLog(cross_amplitude)
file = open('./1/CrossAmplitude.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(norm))

In [5]:
# 1.2
shiftIFT = oneDomainIFFT(img.shift(crossFT, 20, 30))
w, h, contrast = contrastStretching(shiftIFT)
file = open('./1/Cross_Shift.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(contrast))

In [6]:
# 1.3
rotateIFT = oneDomainIFFT(img.rotate(img.toPolarform(crossFT), 30))
w, h, contrast = contrastStretching(rotateIFT)
file = open("./1/Cross_rotate.pgm", "wb")
writePGM(file, w, h, depth, img.flatten(contrast))

In [9]:
# 1.4
downsampling = img.downScale(cross, 100, 100)
downFT = img.inverse(downsampling)
img.padding(downFT,28)
downFT = img.inverse(downFT)

downFT = fft(downFT)
down_amp, down_phase = extractAmplitudeAndPhase(downFT)

w, h, norm = normalizeLog(down_amp)
file = open('./1/Cross_Down_Sampling_Amp.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(norm))

w, h, contrast = contrastStretching(down_phase)
file = open('./1/Cross_Down_Sampling_Phase.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(contrast))

In [8]:
cross_ifft_phase = phaseIFFT(cross_phase)
w, h, contrast = contrastStretching(cross_ifft_phase)
file = open('./1/Cross_ifft_phase.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(contrast))

In [9]:
cross_ifft_amp = ampIFFT(cross_amplitude)
w, h, norm = contrastStretching(cross_ifft_amp)
file = open('./1/Cross_ifft_amplitude.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(norm))

In [12]:
# 1.6
file = open('./1/Lenna.pgm', 'rb')
depth, lenna = readPGM(file)

lennaFT = fft(lenna)
lenna_amplitude, lenna_phase = extractAmplitudeAndPhase(lennaFT)

lenna_ifft_amp = ampIFFT(lenna_amplitude)
lenna_ifft_phase = phaseIFFT(lenna_phase)

w, h, norm = normalizeLog(lenna_amplitude)
file = open('./1/LennaAmplitude.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(norm))

w, h, contrast = contrastStretching(lenna_ifft_phase)
file = open('./1/Lenna_ifft_phase.pgm', 'wb')
writePGM(file, w, h, depth, img.flatten(contrast))